In [2]:
import substrateinterface
from substrateinterface import SubstrateInterface, Keypair
from substrateinterface.exceptions import SubstrateRequestException

substrate = SubstrateInterface(
    url="ws://127.0.0.1:11946",
    ss58_format=42,
    type_registry_preset='kusama'
)

In [3]:
# Function to make extrinsic calls
def make_call(call_module, call_function, call_params, keypair, wait_for_inclusion = True, wait_for_finalization = False):
    call = substrate.compose_call(
        call_module=call_module,
        call_function=call_function,
        call_params=call_params
    )

    extrinsic = substrate.create_signed_extrinsic(call=call, keypair=keypair)

    try:
        receipt = substrate.submit_extrinsic(extrinsic, wait_for_inclusion=wait_for_inclusion, wait_for_finalization=wait_for_finalization)
        print("Extrinsic '{}' sent and included in block '{}'".format(receipt.extrinsic_hash, receipt.block_hash))

    except SubstrateRequestException as e:
        print("Failed to send: {}".format(e))
    return receipt

In [4]:
alice = Keypair.create_from_uri('//Alice')
bob = Keypair.create_from_uri('//Bob')

In [10]:
make_call("Msa", "create", {}, alice)
make_call("Msa", "create", {}, bob)

Extrinsic '0xfd83b6e90f249634e0b9638cdbd609c8755016ff34627cf9dd3aea98ba7fa50f' sent and included in block '0x634f393ca31cbe44af3d51d3c83c84626cb8baeb16c7f743edce2ec90d2ddf31'
Extrinsic '0xa1b5620eb7f427f0e8c95448a3acdd802eb502a4eae028cbed4b448597efc2ac' sent and included in block '0x4873b33b3182ff4cacda06987a5bc4db1adffb1cbd231c084094042dcec65733'


# Create MSA for brand new account
Use this when user wants control of thier keys and will pay for their own transactions

In [11]:
# Create never before used wallet
new_wallet = Keypair.create_from_uri('//ChangeThisNamefgnf', crypto_type=substrateinterface.KeypairType.SR25519)

In [12]:
# Give wallet some tokens to it can make MSA
# Must be at least 1 unit
one_unit = 1000000000000
receipt = make_call("Balances", "transfer", {"dest": new_wallet.ss58_address, "value": 10 * one_unit}, alice)
receipt.error_message

Extrinsic '0x6694a5b75418cc232f00f44b30b96e40fb7bf2917a8e8b3adc4a58af5d2e9523' sent and included in block '0x727a5ef0892cffce809c21f3cf5c6d64430241055c487844906eaddc8e6fc96d'


In [13]:
def get_msa_id(wallet):
    msa_key = substrate.query(
        module='Msa',
        storage_function='KeyInfoOf',
        params=[wallet.ss58_address]
    )

    if msa_key == None:
        make_call("Msa", "create", {}, wallet)
        msa_key = substrate.query(
            module='Msa',
            storage_function='KeyInfoOf',
            params=[wallet.ss58_address]
        )

    msa_id = msa_key['msa_id'].decode()
    return msa_id

In [14]:
msa_id = get_msa_id(new_wallet)

Extrinsic '0x457c28dd5c99f79a8ec20c604ab706dd248d6f3c64744a313199118f15edce57' sent and included in block '0x6de81b76bdc289189d779f84fd49b4da0c002fa01122bf1e3612899cf6b3d0ab'


## Delegate MSA to a provider

In [15]:
payload_raw = { "authorized_msa_id": msa_id, "permission": 0 }

In [16]:
def get_signature(payload, signer):
    # encode payload using SCALE
    # I found scale_info from "substrate.metadata_decoder"
    payload_encoded = substrate.encode_scale(type_string='scale_info::8', value=payload['authorized_msa_id']) + \
                            substrate.encode_scale(type_string='scale_info::2', value=payload['permission'])

    # Payload must be wrapped in theses Bytes objects
    payload_encoded = "<Bytes>".encode() + payload_encoded.data + "</Bytes>".encode()

    # The provider address signs the payload, so in this case alice
    return signer.sign(payload_encoded)

In [17]:
signature = get_signature(payload_raw, alice)

In [18]:
# parameters for call function
call_params = {
    "provider_key": alice.ss58_address,
    "proof": {"Sr25519": "0x" + signature.hex()},
    "add_provider_payload": payload_raw
}

receipt = make_call("Msa", "add_provider_to_msa", call_params, new_wallet)
print(receipt.error_message)

Extrinsic '0x186476870d51e6a7a1a407e208ceca6b3d698b9ee55f27edac13bf0628bd90fb' sent and included in block '0x2071c6d2e043a54684d05c4a459e84c2f00c695186e7c79c4d87377f0540c642'
None


# Following code is run instead of above if user doesn't want control of keys and allows provider to pay for transactions

In [19]:
# Create never before used wallet
new_wallet = Keypair.create_from_uri('//sdvsbsdfghdfdvsvs', crypto_type=substrateinterface.KeypairType.SR25519)

In [20]:
# get provider msa id, so alice in this case
provider_msa_id = get_msa_id(alice)

In [21]:
payload_raw = { "authorized_msa_id": provider_msa_id, "permission": 0 }

# This time new_wallet signs payload and therefore doesn't need to pay any gas
signature = get_signature(payload_raw, new_wallet)

In [22]:
call_params = {
    "delegator_key": new_wallet.ss58_address,
    "proof": {"Sr25519": "0x" + signature.hex()},
    "add_provider_payload": payload_raw
}

# provider signs this
receipt = make_call("Msa", "create_sponsored_account_with_delegation", call_params, alice)
print(receipt.error_message)

Extrinsic '0x6117563e8b1abd4a973da40b9f8be2a38cc6c1d7b1681504e98b9e52215cbf7a' sent and included in block '0x3938843f11f07e499facefdb3aa65ec11f63f1bc6e1ee7db3589337e50240e53'
None


In [26]:
wallets_msa_id = get_msa_id(new_wallet)

# Create Schema and add Messages

In [23]:
# define schema and then check if it exists already, if not, then mint it
# We will probably turn schema check into api call to save time

schema = "subreddit,author,title,selftext,url,is_nsfw,dsvs"

schema_count = substrate.query(
    module='Schemas',
    storage_function='SchemaCount',
    params=[]
).value

schemaId = -1
for i in range(1, schema_count+1):
    schemaTemp = substrate.query(
        module='Schemas',
        storage_function='Schemas',
        params=[i]
    )
    if schemaTemp == schema:
        schemaId = i
        print(schemaTemp.value)
        break
    
if schemaId == -1:
    receipt = make_call("Schemas", "register_schema", {"schema": schema}, alice)
    for event in receipt.triggered_events:
        event = event.decode()
        if event['event']['event_id'] == 'SchemaRegistered':
            schemaId = event['event']['attributes'][1]

print(schemaId)

Extrinsic '0x4d4666981bf5ec922b194963282ef83205f899e3dc003ecf5765411d61271465' sent and included in block '0x2c121167200fd25c2e95126fb2f068f76fc96734306babc62a1caaba2a2615c8'
1


In [24]:
substrate.query(
    module='Balances',
    storage_function='Account',
    params=[new_wallet.ss58_address]
)

<scale_info::5(value={'free': 0, 'reserved': 0, 'misc_frozen': 0, 'fee_frozen': 0})>

In [32]:
message = "AskReddit,LazarShockX,What are some good ice breaker questions?,,https://www.reddit.com/r/AskReddit/comments/v5ihpk/what_are_some_good_ice_breaker_questions/,False"

call_params = {
    "on_behalf_of": wallets_msa_id,
    "schema_id": schemaId,
    "message": message
}
receipt = make_call("Messages", "add", call_params, alice, wait_for_inclusion=True)
wallets_msa_id

# result = substrate.subscribe_block_headers(subscription_handler)

Extrinsic '0x112d4da90949583f9abddc409b510e4f3f83879624fcdad815c60067fc239e2e' sent and included in block '0x3e725c4ddca65027c3625d4cd76963cc3789245c56bac8c6095ffbba19a6a813'


4

In [28]:
receipt.triggered_events

[<scale_info::16(value={'phase': 'ApplyExtrinsic', 'extrinsic_idx': 2, 'event': {'event_index': '0a08', 'module_id': 'Balances', 'event_id': 'Withdraw', 'attributes': ('5GrwvaEF5zXb26Fz9rcQpDWS57CtERHpNehXCPcNoHGKutQY', 2824543777)}, 'event_index': 10, 'module_id': 'Balances', 'event_id': 'Withdraw', 'attributes': ('5GrwvaEF5zXb26Fz9rcQpDWS57CtERHpNehXCPcNoHGKutQY', 2824543777), 'topics': []})>,
 <scale_info::16(value={'phase': 'ApplyExtrinsic', 'extrinsic_idx': 2, 'event': {'event_index': '0a07', 'module_id': 'Balances', 'event_id': 'Deposit', 'attributes': ('5GrwvaEF5zXb26Fz9rcQpDWS57CtERHpNehXCPcNoHGKutQY', 2545158)}, 'event_index': 10, 'module_id': 'Balances', 'event_id': 'Deposit', 'attributes': ('5GrwvaEF5zXb26Fz9rcQpDWS57CtERHpNehXCPcNoHGKutQY', 2545158), 'topics': []})>,
 <scale_info::16(value={'phase': 'ApplyExtrinsic', 'extrinsic_idx': 2, 'event': {'event_index': '0000', 'module_id': 'System', 'event_id': 'ExtrinsicSuccess', 'attributes': {'weight': 125163000, 'class': 'Norma

# Query data
The following use substrate's rpc_request function to call rpc methods that are implemented for the pallets
This doesn't seem to be implemented by the polkadot or substrate frontends so some code might need to be changed

In [5]:
substrate.get_block()['header']['number']

19599

In [6]:
params = [
    schemaId,
    {
        "page_size": 10000,
        "from_block": 0,
        "to_block": 10000,
        "from_index": 1,
    }
]

data = substrate.rpc_request(
    method='messages_getBySchema',
    params=params,
)
data

NameError: name 'schemaId' is not defined

In [357]:
# data can be converted back to original string
bytes.fromhex(data['result']['content'][0]['data'][2:]).decode()

'AskReddit,LazarShockX,What are some good ice breaker questions?,,https://www.reddit.com/r/AskReddit/comments/v5ihpk/what_are_some_good_ice_breaker_questions/,False'

In [327]:
provider_msa_id

3

In [325]:
alice.ss58_address

'5GrwvaEF5zXb26Fz9rcQpDWS57CtERHpNehXCPcNoHGKutQY'

In [8]:
params = [1]

substrate.rpc_request(
    method='msa_getMsaKeys',
    params=params,
)

{'jsonrpc': '2.0',
 'result': [{'expired': 0,
   'key': '5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty',
   'msa_id': 1,
   'nonce': 0}],
 'id': 7}

In [322]:
params = [alice.ss58_address]

substrate.rpc_request(
    method='msa_getMsaId',
    params=params,
)

{'jsonrpc': '2.0', 'result': 3, 'id': 822}

In [330]:
params = [list(range(12)), 3]

substrate.rpc_request(
    method='msa_checkDelegations',
    params=params,
)

{'jsonrpc': '2.0',
 'result': {'0': False,
  '1': False,
  '10': False,
  '11': True,
  '2': True,
  '3': False,
  '4': False,
  '5': False,
  '6': True,
  '7': True,
  '8': True,
  '9': False},
 'id': 830}

In [332]:
params = [2]

substrate.rpc_request(
    method='schemas_getBySchemaId',
    params=params,
)

{'jsonrpc': '2.0',
 'result': {'data': 'subreddit,author,title,selftext,url,is_nsfw,dsvs',
  'schema_id': 2},
 'id': 831}

In [338]:
# params = [_,'subreddit,author,title,selftext,url,is_nsfw,dsvs'.encode().hex()]

# substrate.rpc_request(
#     method='schemas_checkSchemaValidity',
#     params=params,
# )

# Below is test code

In [215]:
def subscription_handler(obj, update_nr, subscription_id):

    print(f"New block #{obj['header']['number']} produced b")
    print(obj)

    if update_nr > 10:
        return {'message': 'Subscription will cancel when a value is returned', 'updates_processed': update_nr}


# result = substrate.subscribe_block_headers?

In [281]:
t = substrate.get_block(substrate.get_block_hash(2662))

In [291]:
t = substrate.get_runtime_events(substrate.get_block_hash(2662))

In [292]:
t['result'][0]

{'phase': 'Initialization',
 'extrinsic_idx': None,
 'event': {'event_index': '2300',
  'module_id': 'Messages',
  'event_id': 'MessagesStored',
  'attributes': (2, 2661, 4)},
 'event_index': 35,
 'module_id': 'Messages',
 'event_id': 'MessagesStored',
 'attributes': (2, 2661, 4),
 'topics': []}

In [293]:
t['result']

[{'phase': 'Initialization',
  'extrinsic_idx': None,
  'event': {'event_index': '2300',
   'module_id': 'Messages',
   'event_id': 'MessagesStored',
   'attributes': (2, 2661, 4)},
  'event_index': 35,
  'module_id': 'Messages',
  'event_id': 'MessagesStored',
  'attributes': (2, 2661, 4),
  'topics': []},
 {'phase': 'ApplyExtrinsic',
  'extrinsic_idx': 0,
  'event': {'event_index': '0000',
   'module_id': 'System',
   'event_id': 'ExtrinsicSuccess',
   'attributes': {'weight': 0, 'class': 'Mandatory', 'pays_fee': 'Yes'}},
  'event_index': 0,
  'module_id': 'System',
  'event_id': 'ExtrinsicSuccess',
  'attributes': {'weight': 0, 'class': 'Mandatory', 'pays_fee': 'Yes'},
  'topics': []},
 {'phase': 'ApplyExtrinsic',
  'extrinsic_idx': 1,
  'event': {'event_index': '0000',
   'module_id': 'System',
   'event_id': 'ExtrinsicSuccess',
   'attributes': {'weight': 155247000,
    'class': 'Mandatory',
    'pays_fee': 'Yes'}},
  'event_index': 0,
  'module_id': 'System',
  'event_id': 'Extri

In [267]:
i = 0
block_hash = substrate.get_block_hash(i)
while (substrate.block_hash != block_hash):
    for event in substrate.get_runtime_events(block_hash)['results']:
        if event['event_id'] == 'MessagesStored':
            
    block_hash = substrate.get_block_hash(i)

'0x0dcab630b48836b3eef826e03d2e62cc29ed8b719abb25a25a28d4398a234be3'

In [113]:
substrate.query_map('System', 'Events', page_size=200, max_results=400)

ValueError: Given storage function is not a map

In [ ]:
substrate.query(
    module='System',
    storage_function='Events',
    params=[],
    block_hash=substrate.get_block_hash(2)
)

In [33]:
substrate.get_block("0x9dbe5e6b82ac046ddd30b1a3c99160f53a8380ca4b461c93b9335273d8b0a066")

{'extrinsics': [<GenericExtrinsic(value={'extrinsic_hash': None, 'extrinsic_length': 2560, 'call': {'call_index': '0x0100', 'call_function': 'set_validation_data', 'call_module': 'ParachainSystem', 'call_args': [{'name': 'data', 'type': 'ParachainInherentData', 'value': {'validation_data': {'parent_head': '0x663b3edc974b0210f161b9fb5492f3cdaef3c9dac650218e574bc9bb43be02adf90f489bfe3ff1d03edc8fab3c5f7759003b27d77ef463e4d775baff7be04612b0ff5c0eb4aa70a802f08012174e84113eceac3d4526707e12325253cd31a1c0014d080661757261203c2538080000000005617572610101685eba937ff8390c02ab8a3ec30682c1f8068b7d44887cf436bfb844b0e46b60f5585870c7db2f841926855287cae272d17706ce8eac9dc94207659710833480', 'relay_parent_number': 4281, 'relay_parent_storage_root': '0xd06b3a3da61d34c2b7b26a336a62ae48f838bd14bbffc761f0c70526aadacfee', 'max_pov_size': 5242880}, 'relay_chain_state': {'trie_nodes': ['0x5f04b49d95320d9021994c850f25b8e3851d03000030000080000008000000000010000000100005000000050000000200000002000000000050000000100

In [244]:
substrateinterface.utils.hasher.xxh128("Sudo".encode()) + substrateinterface.utils.hasher.xxh128("Key".encode())

('5c0d1176a568c1f92944340dbfed9e9c530ebca703c85910e7164cb7d1c9e47b',
 '50a63a871aced22e88ee6466fe5aa5d9')

In [237]:
substrate.rpc_request?

In [302]:
provider_msa_id

4

In [308]:
schemaId

2

In [310]:
params = [
    2,
    {
        "page_size": 10000,
        "from_block": 0,
        "to_block": 10000,
        "from_index": 1,
    }
#     substrateinterface.utils.hasher.xxh128("Messages".encode()) + substrateinterface.utils.hasher.xxh128("BlockMessages".encode()),
#     substrate.get_block_hash(2)
]

substrate.rpc_request(
    method='messages_getBySchema',
    params=params,
)

{'jsonrpc': '2.0',
 'result': {'content': [{'block_number': 2366,
    'data': '0x41736b5265646469742c4c617a617253686f636b582c576861742061726520736f6d6520676f6f642069636520627265616b6572207175657374696f6e733f2c2c68747470733a2f2f7777772e7265646469742e636f6d2f722f41736b5265646469742f636f6d6d656e74732f76356968706b2f776861745f6172655f736f6d655f676f6f645f6963655f627265616b65725f7175657374696f6e732f2c46616c7365',
    'index': 0,
    'msa_id': 9,
    'signer': '5DJhKK6JJzZHcF5V1C4e9noRHcid3C73SRW7xRCnSvNNJa3v'},
   {'block_number': 2374,
    'data': '0x41736b5265646469742c4c617a617253686f636b582c576861742061726520736f6d6520676f6f642069636520627265616b6572207175657374696f6e733f2c2c68747470733a2f2f7777772e7265646469742e636f6d2f722f41736b5265646469742f636f6d6d656e74732f76356968706b2f776861745f6172655f736f6d655f676f6f645f6963655f627265616b65725f7175657374696f6e732f2c46616c7365',
    'index': 0,
    'msa_id': 9,
    'signer': '5DJhKK6JJzZHcF5V1C4e9noRHcid3C73SRW7xRCnSvNNJa3v'},
   {'block_number': 

In [224]:
s

In [351]:
t = data['result']['content'][0]['data']

In [356]:
bytes.fromhex(data['result']['content'][0]['data'][2:]).decode()

'AskReddit,LazarShockX,What are some good ice breaker questions?,,https://www.reddit.com/r/AskReddit/comments/v5ihpk/what_are_some_good_ice_breaker_questions/,False'

In [354]:
bytes.fromhex(message.encode().hex()).decode()

'AskReddit,LazarShockX,What are some good ice breaker questions?,,https://www.reddit.com/r/AskReddit/comments/v5ihpk/what_are_some_good_ice_breaker_questions/,False'

In [348]:
message.encode().hex()

'41736b5265646469742c4c617a617253686f636b582c576861742061726520736f6d6520676f6f642069636520627265616b6572207175657374696f6e733f2c2c68747470733a2f2f7777772e7265646469742e636f6d2f722f41736b5265646469742f636f6d6d656e74732f76356968706b2f776861745f6172655f736f6d655f676f6f645f6963655f627265616b65725f7175657374696f6e732f2c46616c7365'

In [24]:
substrate.chain

BrokenPipeError: [Errno 32] Broken pipe